In [47]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from tensorboardX import SummaryWriter

import gym
# import roboschool
import sys

#%%

import metaworld
import wandb
from tqdm import tqdm



class MLP(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(in_dim, hid_dim)
        self.l2 = nn.Linear(hid_dim, hid_dim)
        self.l3 = nn.Linear(hid_dim, out_dim)
    
    def forward(self, x):
#         print(x)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l3(x)
        return x
    

In [38]:
class ReplayBuffer(object):
    """Buffer to store tuples of experience replay"""

    def __init__(self, max_size=1000000):
        """
        Args:
            max_size (int): total amount of tuples to store
        """

        self.storage = []
        self.max_size = max_size
        self.ptr = 0

    def add(self, data):
        """Add experience tuples to buffer

        Args:
            data (tuple): experience replay tuple
        """

        if len(self.storage) == self.max_size:
            self.storage[int(self.ptr)] = data
            self.ptr = (self.ptr + 1) % self.max_size
        else:
            self.storage.append(data)

    def sample(self, batch_size):
        """Samples a random amount of experiences from buffer of batch size

        Args:
            batch_size (int): size of sample
        """

        ind = np.random.randint(0, len(self.storage), size=batch_size)
        states, actions, next_states, rewards, dones = [], [], [], [], []

        for i in ind:
            s, a, s_, r, d = self.storage[i]
            states.append(np.array(s, copy=False))
            actions.append(np.array(a, copy=False))
            next_states.append(np.array(s_, copy=False))
            rewards.append(np.array(r, copy=False))
            dones.append(np.array(d, copy=False))

        return np.array(states), np.array(actions), np.array(next_states), np.array(rewards).reshape(-1, 1), np.array(
            dones).reshape(-1, 1)

    def save(self):
        np_buffer = np.asarray(self.storage)
        with open('replaybuffer' + str(env._last_rand_vec[0]) + '_' + str(env._last_rand_vec[1]) + '.npy', 'wb') as f:
            print("Saving replay buffer in ", f)
            np.save(f, np_buffer)
            
    def load(self, filename):
        with open(filename, 'rb') as f:
            self.storage = np.load(f, allow_pickle=True)

In [39]:
class Actor(nn.Module):
    """Initialize parameters and build model.
        Args:
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            max_action (float): highest action to take
            seed (int): Random seed
            h1_units (int): Number of nodes in first hidden layer
            h2_units (int): Number of nodes in second hidden layer

        Return:
            action output of network with tanh activation
    """

    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()

        self.l1 = nn.Linear(state_dim, 100)
        self.l2 = nn.Linear(100, 100)
        self.l3 = nn.Linear(100, action_dim)

        self.max_action = max_action

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.max_action * torch.tanh(self.l3(x))
        return x
    
    def load(self, filename="best_avg"):
        state_dict = torch.load(filename)
        for el in state_dict.keys():
            e = state_dict[el]
            print(e.shape)
        self.load_state_dict(state_dict)

    

In [30]:
state_dim = 39
target_actor1,target_actor2,target_actor3,target_actor4 = Actor(state_dim, 4, 1.0),Actor(state_dim, 4, 1.0),Actor(state_dim, 4, 1.0),Actor(state_dim, 4, 1.0)
# target_actor1.load('models/', '.')
# target_actor2.load('models/', '.')
# target_actor3.load('models/', '.')
target_actor4.load('models/best_actor_0.07_0.89.pth')
target_actors = [target_actor1,target_actor2,target_actor3,target_actor4 ]
for a in target_actors:
    a.eval()

torch.Size([100, 39])
torch.Size([100])
torch.Size([100, 100])
torch.Size([100])
torch.Size([4, 100])
torch.Size([4])


In [9]:
r1, r2, r3, r4 = ReplayBuffer(), ReplayBuffer(), ReplayBuffer(), ReplayBuffer()

r1.load('replaybuffers/replaybuffer-0.09_0.86.npy')
r2.load('replaybuffers/replaybuffer-0.09_0.89.npy')
r3.load('replaybuffers/replaybuffer0.07_0.86.npy')
r4.load('replaybuffers/replaybuffer0.07_0.89.npy')

replay_buffers = [r1,r2,r3,r4]


In [51]:
goal_MLP = MLP(2, 100, 4)
state_MLP = MLP(39, 100, 4)
goal_optimizer = torch.optim.Adam(goal_MLP.parameters(), lr=1e-3)
state_optimizer = torch.optim.Adam(state_MLP.parameters(), lr=1e-3)

In [53]:
iterations = 1000000
batch_size = 100
goals = [[-0.09, 0.86], [-0.09, 0.89], [0.07, 0.86], [0.07, 0.89]]
for it in range(iterations):
    for goal_idx in range(4):
        goal_idx = 3
        goal = torch.Tensor(goals[goal_idx])
        s, a, next_s, r, d = replay_buffers[goal_idx].sample(batch_size)
#         print(s)
        loss = 0
        for state in s:
            ss = torch.Tensor(state)
#             print(ss, ss.shape)
            target_action = target_actors[goal_idx](ss)
    #         print(target_action)
            pred_action = torch.mul(goal_MLP(goal), state_MLP(ss))
    
            loss += F.mse_loss(pred_action, target_action)
        print("loss:",loss.item())
        
        goal_MLP.zero_grad()
        state_MLP.zero_grad()
        
        loss.backward()
        
        goal_optimizer.step()
        state_optimizer.step()
        

loss: 32.3123893737793
loss: 25.91537094116211
loss: 25.413902282714844
loss: 23.486955642700195
loss: 24.094789505004883
loss: 23.845666885375977
loss: 22.889659881591797
loss: 23.448060989379883
loss: 21.976669311523438
loss: 26.98719596862793
loss: 22.089059829711914
loss: 27.10405158996582
loss: 24.046409606933594
loss: 26.034502029418945
loss: 26.887184143066406
loss: 17.20363998413086
loss: 26.22256851196289
loss: 25.337697982788086
loss: 20.940895080566406
loss: 31.90908432006836
loss: 24.17228126525879
loss: 22.862823486328125
loss: 22.9405574798584
loss: 23.696990966796875
loss: 24.580900192260742
loss: 23.87381362915039
loss: 24.508468627929688
loss: 18.712753295898438
loss: 24.292552947998047
loss: 24.097267150878906
loss: 26.496665954589844
loss: 30.565288543701172
loss: 28.242158889770508
loss: 23.8030948638916
loss: 28.86391830444336
loss: 22.95073699951172
loss: 23.637338638305664
loss: 27.148216247558594
loss: 19.926971435546875
loss: 21.624292373657227
loss: 19.4784259

loss: 15.30200481414795
loss: 13.187582969665527
loss: 12.251951217651367
loss: 18.833782196044922
loss: 14.212766647338867
loss: 16.307662963867188
loss: 17.87514305114746
loss: 17.19029426574707
loss: 22.209125518798828
loss: 16.16472625732422
loss: 15.672986030578613
loss: 12.88020133972168
loss: 16.049041748046875
loss: 15.531697273254395
loss: 13.977487564086914
loss: 11.229846000671387
loss: 17.422677993774414
loss: 13.3489990234375
loss: 13.817667007446289
loss: 15.638306617736816
loss: 12.922551155090332
loss: 12.722881317138672
loss: 14.314831733703613
loss: 13.897209167480469
loss: 14.652909278869629
loss: 14.951214790344238
loss: 16.752962112426758
loss: 12.275784492492676
loss: 11.492067337036133
loss: 11.0024995803833
loss: 12.485715866088867
loss: 13.81782341003418
loss: 12.256061553955078
loss: 11.364226341247559
loss: 14.796730995178223
loss: 14.494449615478516
loss: 12.267433166503906
loss: 14.198847770690918
loss: 14.384203910827637
loss: 11.203615188598633
loss: 19.4

loss: 8.57304573059082
loss: 12.30463981628418
loss: 7.770837783813477
loss: 11.67042350769043
loss: 8.624429702758789
loss: 8.713440895080566
loss: 9.565385818481445
loss: 13.343927383422852
loss: 7.351375102996826
loss: 11.804169654846191
loss: 7.672314167022705
loss: 10.721632957458496
loss: 13.61758041381836
loss: 11.009604454040527
loss: 10.632195472717285
loss: 11.494012832641602
loss: 9.117975234985352
loss: 9.32942008972168
loss: 11.738771438598633
loss: 5.565145969390869
loss: 11.719050407409668
loss: 11.389750480651855
loss: 9.189425468444824
loss: 9.180371284484863
loss: 11.956560134887695
loss: 8.55887222290039
loss: 8.816642761230469
loss: 11.342386245727539
loss: 8.305708885192871
loss: 9.516539573669434
loss: 11.414201736450195
loss: 11.515995979309082
loss: 11.055933952331543
loss: 7.994706153869629
loss: 10.158622741699219
loss: 11.381184577941895
loss: 9.865621566772461
loss: 11.167745590209961
loss: 9.019479751586914
loss: 9.954532623291016
loss: 9.662182807922363
lo

loss: 5.787863731384277
loss: 6.1669721603393555
loss: 11.515057563781738
loss: 8.390443801879883
loss: 8.297569274902344
loss: 8.369474411010742
loss: 8.908973693847656
loss: 8.469635009765625
loss: 6.014575958251953
loss: 6.455484867095947
loss: 9.552077293395996
loss: 6.543067932128906
loss: 6.7250847816467285
loss: 11.047636985778809
loss: 6.961186408996582
loss: 9.360355377197266
loss: 6.868737697601318
loss: 6.202862739562988
loss: 11.035358428955078
loss: 7.937839031219482
loss: 6.9999542236328125
loss: 8.426614761352539
loss: 11.65091609954834
loss: 6.699492931365967
loss: 9.484058380126953
loss: 9.826491355895996
loss: 5.590217113494873
loss: 6.2920732498168945
loss: 10.476919174194336
loss: 6.173051357269287
loss: 8.829498291015625
loss: 8.154812812805176
loss: 4.0468573570251465
loss: 6.394758224487305
loss: 6.154099464416504
loss: 6.089356899261475
loss: 9.178865432739258
loss: 6.192726135253906
loss: 7.106168746948242
loss: 9.12480354309082
loss: 5.510207176208496
loss: 8.

loss: 4.751997947692871
loss: 7.224465370178223
loss: 5.159514427185059
loss: 4.471591949462891
loss: 8.293102264404297
loss: 5.980741024017334
loss: 4.602952003479004
loss: 5.2347917556762695
loss: 6.480131149291992
loss: 6.686370849609375
loss: 7.642491817474365
loss: 4.981332302093506
loss: 9.541157722473145
loss: 4.582320690155029
loss: 6.966569900512695
loss: 6.471115589141846
loss: 5.031717777252197
loss: 6.976619720458984
loss: 7.178711891174316
loss: 7.811234474182129
loss: 5.639024257659912
loss: 6.29374885559082
loss: 6.073317050933838
loss: 7.074146747589111
loss: 6.284794807434082
loss: 5.876015663146973
loss: 5.388246059417725
loss: 7.1530022621154785
loss: 7.298311233520508
loss: 7.110743045806885
loss: 6.095985412597656
loss: 4.441765308380127
loss: 7.778897762298584
loss: 6.5496392250061035
loss: 9.16098690032959
loss: 8.63189697265625
loss: 7.183173179626465
loss: 7.989146709442139
loss: 7.142419338226318
loss: 7.5545549392700195
loss: 6.546630382537842
loss: 9.0430841

...

loss: 2.261042594909668
loss: 4.1403398513793945
loss: 2.2351608276367188
loss: 2.1657941341400146
loss: 4.6269097328186035
loss: 2.048332691192627
loss: 3.7663891315460205
loss: 2.7708401679992676
loss: 2.83075213432312
loss: 2.1821722984313965
loss: 2.203380823135376
loss: 3.7873904705047607
loss: 1.800998568534851
loss: 2.460448741912842
loss: 2.236924171447754
loss: 2.96022367477417
loss: 1.870997667312622
loss: 1.4981085062026978
loss: 4.952810764312744
loss: 2.837630271911621
loss: 3.8912148475646973
loss: 1.2993332147598267
loss: 1.5703485012054443
loss: 2.8993096351623535
loss: 2.500455617904663
loss: 2.612661361694336
loss: 1.663316011428833
loss: 2.7484092712402344
loss: 1.3994367122650146
loss: 2.7158141136169434
loss: 2.206526756286621
loss: 2.884701728820801
loss: 2.4582014083862305
loss: 3.249391794204712
loss: 2.410731077194214
loss: 3.8278844356536865
loss: 1.4766627550125122
loss: 1.7533133029937744
loss: 2.910360097885132
loss: 2.5009355545043945
loss: 2.3681023120880

loss: 3.18528413772583
loss: 2.659663438796997
loss: 3.3458845615386963
loss: 3.2834508419036865
loss: 3.4092371463775635
loss: 3.72103214263916
loss: 2.311171293258667
loss: 3.971342086791992
loss: 2.6163790225982666
loss: 3.597079277038574
loss: 3.171409845352173
loss: 2.982729434967041
loss: 4.925744533538818
loss: 2.9376351833343506
loss: 2.5417420864105225
loss: 3.5799527168273926
loss: 4.844937801361084
loss: 2.305793046951294
loss: 1.6762495040893555
loss: 2.136967420578003
loss: 4.284861087799072
loss: 4.081762790679932
loss: 2.7115283012390137
loss: 2.4349780082702637
loss: 2.7481532096862793
loss: 2.5219218730926514
loss: 4.3184685707092285
loss: 3.3863182067871094
loss: 1.7267032861709595
loss: 3.8356151580810547
loss: 3.1730847358703613
loss: 2.6650712490081787
loss: 1.6458522081375122
loss: 2.9609122276306152
loss: 1.8420751094818115
loss: 3.8889636993408203
loss: 2.5465784072875977
loss: 3.4541149139404297
loss: 2.4387543201446533
loss: 2.238055467605591
loss: 2.936805963

loss: 2.3605737686157227
loss: 2.2106142044067383
loss: 3.794987201690674
loss: 4.244705677032471
loss: 2.8473258018493652
loss: 3.050820827484131
loss: 3.3342549800872803
loss: 2.411001443862915
loss: 3.273205041885376
loss: 3.1808745861053467
loss: 2.0343310832977295
loss: 2.7380282878875732
loss: 2.3286938667297363
loss: 2.6480789184570312
loss: 4.78369140625
loss: 2.3457908630371094
loss: 2.564758777618408
loss: 2.1349923610687256
loss: 3.6477952003479004
loss: 2.761059045791626
loss: 2.073784351348877
loss: 1.6797375679016113
loss: 2.6585099697113037
loss: 4.578192710876465
loss: 3.138427495956421
loss: 4.053702354431152
loss: 2.719697952270508
loss: 2.163130521774292
loss: 2.787029266357422
loss: 2.936006546020508
loss: 3.1295523643493652
loss: 2.053662061691284
loss: 2.2370855808258057
loss: 3.975830316543579
loss: 2.1980128288269043
loss: 3.249183416366577
loss: 2.748521566390991
loss: 1.8350566625595093
loss: 2.6564884185791016
loss: 2.1978023052215576
loss: 3.0413358211517334

loss: 2.9108657836914062
loss: 4.42857551574707
loss: 3.3746488094329834
loss: 1.65840744972229
loss: 1.7264790534973145
loss: 3.2930588722229004
loss: 1.9020366668701172
loss: 3.249683141708374
loss: 1.9183766841888428
loss: 2.8477847576141357
loss: 2.0400140285491943
loss: 4.094210624694824
loss: 4.208926200866699
loss: 3.011758327484131
loss: 2.2679338455200195
loss: 2.65163254737854
loss: 3.0986547470092773
loss: 1.9276992082595825
loss: 1.5190767049789429
loss: 3.7219481468200684
loss: 2.40714168548584
loss: 2.511596441268921
loss: 2.8160150051116943
loss: 3.425743579864502
loss: 2.668327808380127
loss: 1.181393027305603
loss: 2.5377047061920166
loss: 2.0420682430267334
loss: 2.676826000213623
loss: 3.2462823390960693
loss: 2.448737382888794
loss: 2.0329277515411377
loss: 2.450591802597046
loss: 1.6434639692306519
loss: 2.8705544471740723
loss: 2.099034070968628
loss: 1.732547402381897
loss: 1.624420166015625
loss: 4.160811901092529
loss: 1.4415639638900757
loss: 2.273481130599975

KeyboardInterrupt: 